Importing libraries needed to fetch web-site content and create data file respectively.

In [16]:
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from urllib.request import Request as Req
from bs4 import BeautifulSoup as soup

Creating list of locations for vacancies search. Due to architecture of website Monster.com it shows maximum of 260 open position per location. Hence, to increase amount of data fetched I have introduced 10 different locations.

In [17]:
search_locations=['New-York__2C-NY', 'New-York', 'San-Diego__2C-CA', 'Texas', 'Los-Angeles__2C-CA', 'Bay-Area', 
                  'Chicago__2C-IL', 'Detroit', 'Ohio', 'Florida']

Defining method which will be used for constructing URL based on key word variable and location input parameter.

In [18]:
def get_url(loc):
    keyWord = 'Data-Science'
    urlorig='https://www.monster.com/jobs/search/?q='+keyWord+'&where='+loc+'&stpage=1&page=10'
    return urlorig

In cell below I defined two parsers. First one is to read and parse data from list of jobs and second one designed to open URL for each open vacancy and parse it's content for additional details anf job description. First parser calls a second one for each job.

I also on purpose do not fetch any skills during parsing with this version of web scrapper because set of skills is not finite and different skills could be required for various researches. Hence, I save full description which could be later investigated with regards of required skill set. 

In [19]:
titles = []
company_names = []
location_names = []
job_descriptions = []
job_ids = []
refs=[]
compensations = []

global_ind=0

# defining parser to call for each new page
def monster_parser(url):
    global global_ind
    request = Req(url)
    request.add_header('Cache-Control', 'no-cache')
    uClient = uReq(request)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,'html.parser')
    jobs = page_soup.select('section.card-content')
    
    for job in jobs:
        if job.get('data-jobid')!=None:
            global_ind+=1
            summaries = job.div.find_all('div', class_='summary')
            summary=summaries[0]

            try:
                #check if vacancy exist
                tmp_client = uReq(summary.header.h2.a['href'])
                tmp_client.close()

                print(global_ind, summary.header.h2.a['href'])

                #fetch Job ID
                try:
                    job_ids.append(job['data-jobid'])
                except:
                    job_ids.append('NA')

                #fetching URLs
                try:
                    refs.append(summary.header.h2.a['href'])
                except:
                    refs.append('NA')

                #fetch title
                try:
                    titles.append(summary.header.h2.a.text.strip())
                except:
                    titles.append('NA')
                #fetch company
                try:
                    company_names.append(summary.div.span.text)
                except:
                    company_names.append('NA')

    #             locations = summary.find_all('div', class_='location')
    #             location = locations[0]
    #             #fetch location
    #             try:
    #                 location_names.append(location.span.text.strip())
    #             except:
    #                 location_names.append('NA')
                #parse job description and details
                monster_details_parser(summary.header.h2.a['href'])

            except:
                 print('SKIP - ', summary.header.h2.a['href'])
                
def monster_details_parser(url):
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,'html.parser')
    
    descr = page_soup.select('div#JobDescription')
    job_descriptions.append('NA')
    try:
        job_descriptions.append(descr[0].text)
        job_descriptions.pop(-2)
    except:
        job_descriptions.append('NA')
        job_descriptions.pop(-2)
        
    values = page_soup.select('div.mux-job-summary section.summary-section dl.header dd.value')
    location_names.append('NA')
    for value in values:
        if value.parent.dt.text == "Location":
            #fetch location
            try:
                location_names.append(value.text)
                location_names.pop(-2)
            except:
                location_names.append('NA')
                location_names.pop(-2)
    
    comp_values = page_soup.select('div#JobSalary.mux-card.mux-job-cards section.card-content div.row div.col-xs-12.cell div')
    compensations.append('NA')
    try:
        compensations.append(comp_values[0].text.strip())
        compensations.pop(-2)
    except:
        compensations.append('NA')
        compensations.pop(-2)

This cell is used to loop trough search locations and call parser methods for each.

In [20]:
for loc in search_locations:
    monster_parser(get_url(loc))
    #check sizes of arrays betwe locations to ensure possibility to create DF afterwards
    print(len(titles),len(company_names), len(location_names), len(job_ids), len(job_descriptions), len(refs), 
      len(compensations))
    

1 https://job-openings.monster.com/junior-data-scientist-nyc-manhattan-ny-us-visionaire-partners/213759799
2 https://job-openings.monster.com/data-scientist-new-york-city-ny-us-national-debt-relief/213099686
3 https://job-openings.monster.com/data-scientist-nyc-new-york-ny-us-interactive-brokers-llc/213764166
4 https://job-openings.monster.com/head-of-data-science-discovery-new-york-ny-us-facebook/be31d872-e8b3-47dc-9033-76140bcebe27
5 https://job-openings.monster.com/product-manager-new-york-ny-us-apex-systems/213794635
6 https://job-openings.monster.com/entity-modeler-new-york-city-ny-us-focus-capital-markets/211908458
7 https://job-openings.monster.com/big-data-rutherford-nj-us-synechron/213043426
8 https://job-openings.monster.com/sr-data-analyst-vc-backed-healthtech-manhattan-ny-us-grow/213787743
9 https://job-openings.monster.com/ux-designer-senior-level-new-york-city-ny-us-the-judge-group/213327753
10 https://job-openings.monster.com/product-designer-ui-ux-new-york-ny-us-clark-d

70 https://job-openings.monster.com/lead-data-scientist-new-york-ny-us-mohawk-group-inc/ab9c1377-8da1-49e0-b137-1d7263de18e5
71 https://job-openings.monster.com/data-scientist-new-york-ny-us-cerberus-capital-management/9bb1e6a8-1eda-499c-bde0-2dd25b037085
72 https://job-openings.monster.com/data-scientist-new-york-ny-us-career-search-partners/97627da8-8547-4154-ad62-387708a53f3f
73 https://job-openings.monster.com/data-scientist-new-york-ny-us-smith-arnold-partners/7d37a194-a1ec-4e0f-a90a-9be5468900f5
74 https://job-openings.monster.com/data-scientist-new-york-ny-us-remedy-partners/568a7d19-1f32-42c7-91e3-d1d2dfc940ae
75 https://job-openings.monster.com/data-scientist-new-york-ny-us-softvision-north-america-uk/5671e3ce-f72c-4409-b052-ef6371eb30a7
76 https://job-openings.monster.com/data-scientist-associate-landau-lab-new-york-ny-us-new-york-genome-center/55665c95-2434-454d-9dcb-c02ca6dd2ca9
77 https://job-openings.monster.com/data-scientist-product-analytics-brooklyn-ny-us-etsy/f018be7

132 https://job-openings.monster.com/data-scientist-manager-sr-manager-credit-fraud-risk-new-york-ny-us-american-express/4d30ec56-117d-4cca-bdc8-337e8dfd06e1
133 https://job-openings.monster.com/data-scientist-eft-analytics-new-york-ny-us-georgia-pacific/c9685071-5419-475a-aae0-51f5c2e9303a
134 https://job-openings.monster.com/senior-data-scientist-new-york-ny-us-park-hudson-international-llc/0be7fe5a-7bb8-4dff-a6a8-6adba9265fec
135 https://job-openings.monster.com/data-scientist-newark-nj-us-implify-inc/43b5f510-bfb8-493e-990f-9594f3539ef8
136 https://job-openings.monster.com/data-scientist-new-york-city-ny-us-lancesoft-inc/938bf049-68f5-4fd7-b614-ff22a8476d48
137 https://job-openings.monster.com/data-scientist-new-york-ny-us-apar-technologies/3c402eb2-217e-4953-a43a-7f7dc6920c92
138 https://job-openings.monster.com/data-scientist-tableau-developer-new-york-city-ny-us-veterans-sourcing-group/9796ae2e-bec9-4928-a814-f8a8c5882a58
139 https://job-openings.monster.com/data-scientist-risk-

198 https://job-openings.monster.com/data-scientist-new-york-ny-us-bank-of-america/e17c8cae-5153-4d60-8327-9829e627ebe6
199 https://job-openings.monster.com/data-scientist-nyc-ny-us-aptask/b99a9476-528f-4b20-b4a8-24b61d80ba83
200 https://job-openings.monster.com/data-scientist-new-york-ny-us-park-hudson-international-llc/081f9824-2a1e-4ec6-89ab-07d2e55b5320
201 https://job-openings.monster.com/data-scientist-tableau-developer-new-york-city-ny-us-veterans-sourcing-group/478c5e9a-6cf8-482a-a82e-c2fcae80c66c
202 https://job-openings.monster.com/data-scientist-new-york-ny-us-the-ashlar-group/56ea2939-c5a0-4ede-97b8-4c1829886b86
203 https://job-openings.monster.com/data-scientist-big-data-engineer-new-york-ny-us-emblemhealth/130338dd-3345-4680-a48d-c1afb8a34c72
204 https://job-openings.monster.com/data-scientist-new-york-ny-us-newyork-presbyterian/52eee08e-4255-4fd8-af49-7bdc14a2716b
205 https://job-openings.monster.com/rmbs-data-scientist-new-york-ny-us-atlantic-partners/5c9a93fd-f1f5-40fc

261 https://job-openings.monster.com/product-manager-new-york-ny-us-apex-systems/213794635
262 https://job-openings.monster.com/entity-modeler-new-york-city-ny-us-focus-capital-markets/211908458
263 https://job-openings.monster.com/business-data-analyst-reston-va-us-aboutweb/213764734
264 https://job-openings.monster.com/sr-data-analyst-vc-backed-healthtech-manhattan-ny-us-grow/213787743
265 https://job-openings.monster.com/ux-designer-senior-level-new-york-city-ny-us-the-judge-group/213327753
266 https://job-openings.monster.com/product-designer-ui-ux-new-york-ny-us-clark-davis-associates/213297367
267 https://job-openings.monster.com/executive-assistant-tech-manhattan-ny-us-beacon-hill-staffing-group-llc/213689351
268 https://job-openings.monster.com/senior-manager-emerging-customer-analytics-coe-new-york-ny-us-pfizer/213257206
269 https://job-openings.monster.com/data-engineering-architect-at-artificial-intelligence-startup-manhattan-ny-us-cypress-group/212765213
270 https://job-ope

...

2066 https://job-openings.monster.com/marketing-technologist-cincinnati-oh-us-procter-gamble-co/8899db7c-9cee-47ce-98ef-d2875595fc6e
2067 https://job-openings.monster.com/data-architect-senior-associate-columbus-oh-us-jpmorgan-chase-co/e7399a37-5f1c-4e67-88f2-2a665848382b
1991 1991 1991 1991 1991 1991 1991
2068 https://job-openings.monster.com/data-scientist-kissimmee-fl-us-ciber-inc/213730238
2069 https://job-openings.monster.com/data-scientist-miami-fl-us-apex-systems/213147975
2070 https://job-openings.monster.com/senior-data-scientist-sales-insights-analytics-boca-raton-fl-us-adt-security/a9127d4a-33a8-44f0-bb3b-72b3b5b9b367
2071 https://job-openings.monster.com/manager-data-science-oldsmar-fl-us-the-nielsen-company-llc/213638755
2072 https://job-openings.monster.com/data-scientist-orlando-fl-us-ciber-inc/213588576
2073 https://job-openings.monster.com/big-data-tampa-fl-us-synechron/213043448
2074 https://job-openings.monster.com/full-stack-developer-oviedo-fl-us-rx-development/212

2134 https://job-openings.monster.com/data-scientist-miami-fl-us-pozent/187cfce7-150f-47d6-b6d5-999c15f85c8c
2135 https://job-openings.monster.com/computer-information-research-data-scientist-orlando-fl-us-qualis-corporation/d24ed319-a5c9-4e39-9626-a3953494be87
2136 https://job-openings.monster.com/data-scientist-tampa-fl-us-wellcare/b23590be-4020-453b-b55f-678f57584dcb
2137 https://job-openings.monster.com/dir-data-science-healthcare-lnr008cq-boca-raton-fl-us-reed-elsevier/02c6cf81-4305-4051-b1d7-d3a3c04347ab
2138 https://job-openings.monster.com/data-science-assistant-gainesville-fl-us-university-of-florida/36f6c9a9-e8d0-4e1d-b866-17c8aed1f496
2139 https://job-openings.monster.com/data-science-machine-learning-engineer-tampa-fl-us-jpmorgan-chase-co/c70bb31e-8d24-41f6-a199-40a6bd11ee69
2140 https://job-openings.monster.com/quantitative-risk-analyst-lead-data-science-tampa-fl-us-usaa/39249fda-2221-4486-b192-8d523038bb6e
2141 https://job-openings.monster.com/manager-data-science-enginee

SKIP -  https://job-openings.monster.com/principal-lead-devops-development-engineer-–-devops-product-development-tampa-fl-us-princeton-information-ltd/69cb8f31-cea2-4566-9dc3-a71ef56848aa
2196 https://job-openings.monster.com/interns-summer-2020-jupiter-fl-us-voloridge-investment-management/392d2afd-99d4-4568-9096-61819629b146
2197 https://job-openings.monster.com/sr-data-architect-lake-mary-fl-us-blue-ocean-ventures/c3cc6d1a-c067-47cd-a219-84133c5d2c14
2198 https://job-openings.monster.com/systems-data-analyst-jacksonville-fl-us-software-resources-inc/a2dab933-c173-4f71-b50d-d45645674329
2199 https://job-openings.monster.com/sr-data-architect-lake-mary-fl-us-aptask/93e8889c-f1a5-417e-81a2-eedf483859d2
2200 https://job-openings.monster.com/data-analyst-consulting-position-tampa-fl-us-the-ashlar-group/57a45931-53ea-4194-925e-d15b4fe8fb58
2201 https://job-openings.monster.com/cib-wholesale-payments-data-insights-dashboards-product-manager-vice-president-tampa-fl-us-jp-morgan/b8099d83-af4

2254 https://job-openings.monster.com/devops-qa-engineer-software-development-engineer-in-test-sde-t-tampa-fl-us-princeton-information-ltd/3a7f5844-7626-4403-b9d7-7689f27d58be
2255 https://job-openings.monster.com/hiring-2020-graduates-jupiter-fl-us-voloridge-investment-management/32906c69-08ff-401c-9273-1105f8fe840e
2256 https://job-openings.monster.com/sr-data-architect-lake-mary-fl-us-blue-ocean-ventures/c2793e49-6d7e-42ec-a461-19aea368c109
2257 https://job-openings.monster.com/business-analyst-with-pega-miami-fl-us-aptask/08f870fd-05a2-4a8b-8e7f-24b461e66d46
2258 https://job-openings.monster.com/licensed-health-insurance-agent-dont-miss-oep-perry-fl-us-assurance/fdb62ad4-6941-4873-aba0-26f97b9bf44d
2259 https://job-openings.monster.com/data-analyst-homestead-fl-us-itility-llc/b99300ae-3f9e-46b4-9057-e91397a7893b
2260 https://job-openings.monster.com/it-security-specialist-us-interactive-resources-llc/213790002
2261 https://job-openings.monster.com/cib-wholesale-payments-data-produc

2312 https://job-openings.monster.com/receptionist-admin-assistant-intake-louisville-ky-us/191937328
2313 https://job-openings.monster.com/licensed-health-insurance-agent-dont-miss-oep-palm-springs-fl-us-assurance/eefea1f0-cd0b-476f-8b69-9b6099765463
2314 https://job-openings.monster.com/cib-data-design-head-of-data-visualization-insights-executive-director-tampa-fl-us-jpmorgan-chase-co/e8beedd6-1143-46fa-b363-7cb917777369
2315 https://job-openings.monster.com/cyber-security-analyst-jacksonville-fl-us-andromeda-systems-incorporated/ce473a58-ea7b-4b05-9a00-ac909f5a2ce8
2316 https://job-openings.monster.com/licensed-health-insurance-agent-dont-miss-oep-indian-rocks-beach-fl-us-assurance/eee715bf-ef5d-42ad-873d-ebfa8de1a29f
2317 https://job-openings.monster.com/cib-wholesale-payments-data-product-delivery-manager-associate-vice-president-tampa-fl-us-jpmorgan-chase-co/08c2fa79-8f15-487e-a016-a5d59b9f22ed
2318 https://job-openings.monster.com/licensed-health-insurance-agent-dont-miss-oep-hi

Next, building dataset based on arrays with data.

In [23]:
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 
     'Job_ID':job_ids,'Job_Description':job_descriptions, 'Link': refs, 'Compensation': compensations}

df1 = pd.DataFrame(d)
results=df1
results.head()

,Job_Title,Company_Name,Location,Job_ID,Job_Description,Link,Compensation
0,Junior Data Scientist - NYC,Visionaire Partners,"Manhattan, NY 10001",213759799,\nData Scientist - NYCExciting opportunity to ...,https://job-openings.monster.com/junior-data-s...,DOE
1,Data Scientist,National Debt Relief,"New York City, NY 10038",213099686,\nWho We’re Looking For:National Debt Relief (...,https://job-openings.monster.com/data-scientis...,NA
2,Data Scientist (NYC),Interactive Brokers LLC,"New York, NY",213764166,\nJob Description and ResponsibilitiesInteract...,https://job-openings.monster.com/data-scientis...,NA
3,Head of Data Science Discovery,Facebook,"New York, NY",be31d872-e8b3-47dc-9033-76140bcebe27,\nCompany is a global community of more than 1...,https://job-openings.monster.com/head-of-data-...,NA
4,Product Manager,Apex Systems,"New York, NY",213794635,\n Job Description Job #: 1016957••Please sen...,https://job-openings.monster.com/product-manag...,NA


Since there are possible overlays between serach locations I am droping duplicates using built-in methods of DataFrame.

In [24]:
print(len(results))
result_unique = results.drop_duplicates()
print(len(result_unique))

2245
2043


Saving data to csv file to be used in further research.

In [25]:
result_unique.to_csv('Monster_job_vacancies.csv')